# Transformers Classification

### 1. Setup and Installation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
sns.set_style('darkgrid')

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback, IntervalStrategy
from transformers import DataCollatorWithPadding
import evaluate
from tqdm.auto import tqdm

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
import random
import time
import os

from dotenv import load_dotenv
ENV_PATH = os.path.join(os.path.dirname(os.getcwd()), '.env')
load_dotenv(ENV_PATH)
access_token = os.environ.get("ACCESS_TOKEN")

import warnings
warnings.filterwarnings('ignore')

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

### 2. Data Preparation and Splitting

In [ ]:
DATA_PATH = os.path.join(os.path.dirname(os.getcwd()), 'data', 'RHMD_Engineered.csv')
df = pd.read_csv(DATA_PATH)

In [ ]:
mentalhealth_df = df[df['subreddit'] == 'mentalhealth'].copy()
df = df[df['subreddit'] != 'mentalhealth'].copy()

In [ ]:
print(f"Number of posts from specific mental health subreddits: {len(df)}")
print(f"Number of general mentalhealth posts to be classified: {len(mentalhealth_df)}")

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(y='subreddit', data=df)
plt.title('Distribution of Posts Across Specific Mental Health Subreddits')
plt.xlabel('Count')
plt.ylabel('Subreddit')
plt.tight_layout()
plt.show()

In [ ]:
df['full_text'] = df['title'] + " [SEP] " + df['text']
mentalhealth_df['full_text'] = mentalhealth_df['title'] + " [SEP] " + mentalhealth_df['text']

In [ ]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['subreddit'])

In [ ]:
for i, subreddit in enumerate(label_encoder.classes_):
    print(f"{subreddit} -> {i}")

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=SEED, stratify=df['subreddit'])

In [ ]:
print(f"\nTraining set size: {len(train_df)}")
print(f"Testing set size: {len(test_df)}")

### 3. Making Dataset Classes for Huggingface Transformers

In [ ]:
class MentalHealthDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['full_text']
        label = self.dataframe.iloc[idx]['label'] if 'label' in self.dataframe.columns else 0

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        encoding = {k: v.squeeze(0) for k, v in encoding.items()}

        return {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
            'labels': torch.tensor(label, dtype=torch.long)
        }

### 4. Model Training Function

In [ ]:
def train_model(model_name, tokenizer, train_dataset, test_dataset, num_labels):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=num_labels,
        ignore_mismatched_sizes=True,
        token=access_token
    )

    # Freeze all parameters
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the last transformer layer and classifier
    if "roberta" in model_name.lower():
        for param in model.roberta.encoder.layer[-1].parameters():
            param.requires_grad = True
    else:
        for param in model.bert.encoder.layer[-1].parameters():
            param.requires_grad = True

    # Unfreeze the classifier
    for param in model.classifier.parameters():
        param.requires_grad = True

    data_collator = DataCollatorWithPadding(tokenizer)

    metrics = evaluate.combine(["precision", "recall", "f1"])

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        return metrics.compute(predictions=predictions, references=labels, average="weighted")

    training_args = TrainingArguments(
        output_dir=f"./results_{model_name.replace('/', '-')}",
        eval_steps = 100,
        logging_steps=100,
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        save_total_limit = 5,
        num_train_epochs=10,
        weight_decay=0.01,
        evaluation_strategy=IntervalStrategy.STEPS,
        load_best_model_at_end=True,
        push_to_hub=False,
        report_to="none"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience=7)]
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return model, eval_results

In [ ]:
def evaluate_model(model, dataset, label_encoder):
    model.eval()
    model.to(device)

    all_preds = []
    all_labels = []

    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs = {
                'input_ids': batch['input_ids'].to(device),
                'attention_mask': batch['attention_mask'].to(device)
            }
            labels = batch['labels'].to(device)

            outputs = model(**inputs)
            logits = outputs.logits

            preds = torch.argmax(logits, dim=-1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)

    report = classification_report(
        all_labels,
        all_preds,
        target_names=label_encoder.classes_,
        digits=4
    )

    cm = confusion_matrix(all_labels, all_preds)

    return accuracy, report, cm, all_preds, all_labels

### 5. Train and Evaluate Model: mental/mental-roberta-base

In [ ]:
model_name = "mental/mental-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)

In [ ]:
train_dataset = MentalHealthDataset(train_df, tokenizer)
test_dataset = MentalHealthDataset(test_df, tokenizer)

In [ ]:
start_time = time.time()
roberta_model, roberta_results = train_model(
    model_name,
    tokenizer,
    train_dataset,
    test_dataset,
    num_labels=len(label_encoder.classes_)
)
roberta_training_time = time.time() - start_time

In [ ]:
roberta_training_time = time.time() - start_time
print(f"Training completed in {roberta_training_time:.2f} seconds")
print(f"Evaluation results: {roberta_results}")

In [ ]:
roberta_accuracy, roberta_report, roberta_cm, roberta_preds, roberta_labels = evaluate_model(
    roberta_model,
    test_dataset,
    label_encoder
)

In [ ]:
print(f"Accuracy: {roberta_accuracy:.4f}")

In [ ]:
print("\nClassification Report:")
print(roberta_report)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(roberta_cm, annot=True, fmt='d', cmap='rocket', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix - RoBERTa Model')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

## 6. Train and Evaluate Model: mental/mental-bert-base-uncased

In [ ]:
model_name = "mental/mental-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)

In [ ]:
train_dataset = MentalHealthDataset(train_df, tokenizer)
test_dataset = MentalHealthDataset(test_df, tokenizer)

In [ ]:
start_time = time.time()
bert_model, bert_results = train_model(
    model_name,
    tokenizer,
    train_dataset,
    test_dataset,
    num_labels=len(label_encoder.classes_)
)
bert_training_time = time.time() - start_time

In [ ]:
print(f"Training completed in {bert_training_time:.2f} seconds")
print(f"Evaluation results: {bert_results}")

In [ ]:
bert_accuracy, bert_report, bert_cm, bert_preds, bert_labels = evaluate_model(bert_model, test_dataset, label_encoder)

In [ ]:
print(f"Accuracy: {bert_accuracy:.4f}")

In [ ]:
print("\nClassification Report:")
print(bert_report)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(bert_cm, annot=True, fmt='d', cmap='rocket', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix - BERT Model')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

In [ ]:
if roberta_accuracy > bert_accuracy:
    best_model_name = "RoBERTa"
    best_preds = roberta_preds
    best_model = roberta_model
    best_tokenizer = AutoTokenizer.from_pretrained("mental/mental-roberta-base")
else:
    best_model_name = "BERT"
    best_preds = bert_preds
    best_model = bert_model
    best_tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")

### 7. Error Analysis

In [ ]:
misclassified_indices = [i for i, (pred, true) in enumerate(zip(best_preds, roberta_labels)) if pred != true]
print(f"Number of misclassified examples: {len(misclassified_indices)} out of {len(test_df)}")

In [ ]:
test_df_reset = test_df.reset_index(drop=True)
misclassified_df = test_df_reset.iloc[misclassified_indices].copy()
misclassified_df['predicted_label'] = [best_preds[i] for i in misclassified_indices]
misclassified_df['predicted_subreddit'] = label_encoder.inverse_transform(misclassified_df['predicted_label'])

In [ ]:
print("\nExamples of misclassifications:")
for i, (idx, row) in enumerate(misclassified_df.head(5).iterrows()):
    print(f"\nExample {i+1}:")
    print(f"True subreddit: {row['subreddit']}")
    print(f"Predicted subreddit: {row['predicted_subreddit']}")
    print(f"Title: {row['title']}")
    print(f"First 150 chars of text: {row['text'][:150]}...")
    print("-" * 80)

In [ ]:
misclass_pairs = misclassified_df.groupby(['subreddit', 'predicted_subreddit']).size().reset_index()
misclass_pairs.columns = ['True Subreddit', 'Predicted Subreddit', 'Count']
misclass_pairs = misclass_pairs.sort_values('Count', ascending=False)

In [ ]:
print("\nMost common misclassification patterns:")
print(misclass_pairs.head(5))

In [ ]:
plt.figure(figsize=(12, 8))
misclass_matrix = pd.crosstab(misclassified_df['subreddit'], misclassified_df['predicted_subreddit'], normalize='index') * 100
sns.heatmap(misclass_matrix, annot=True, fmt='.1f', cmap='rocket')
plt.title('Misclassification Patterns (% of Errors)')
plt.ylabel('True Subreddit')
plt.xlabel('Predicted Subreddit')
plt.tight_layout()
plt.show()

## 8. Classifying Mentalhealth Posts

In [ ]:
mentalhealth_dataset = MentalHealthDataset(mentalhealth_df.assign(label=0), best_tokenizer)

In [ ]:
mentalhealth_loader = DataLoader(mentalhealth_dataset, batch_size=32, shuffle=False)

In [ ]:
best_model.eval()
best_model.to(device)

In [ ]:
mentalhealth_preds = []
with torch.no_grad():
    for batch in tqdm(mentalhealth_loader, desc="Predicting"):
        inputs = {
            'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)
        }

        outputs = best_model(**inputs)
        logits = outputs.logits

        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        probs = torch.softmax(logits, dim=-1).cpu().numpy()

        mentalhealth_preds.extend([(pred, probs[i]) for i, pred in enumerate(preds)])

In [ ]:
mentalhealth_df['predicted_label'] = [pred[0] for pred in mentalhealth_preds]
mentalhealth_df['predicted_subreddit'] = label_encoder.inverse_transform(mentalhealth_df['predicted_label'])

In [ ]:
for i, class_name in enumerate(label_encoder.classes_):
    mentalhealth_df[f'prob_{class_name}'] = [pred[1][i] for pred in mentalhealth_preds]

In [ ]:
mentalhealth_df['confidence'] = [np.max(pred[1]) for pred in mentalhealth_preds]

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(y='predicted_subreddit', data=mentalhealth_df)
plt.title('Distribution of Predicted Subreddits for Mentalhealth Posts')
plt.xlabel('Count')
plt.ylabel('Predicted Subreddit')
plt.tight_layout()
plt.show()

In [ ]:
predicted_counts = mentalhealth_df['predicted_subreddit'].value_counts()
predicted_percentages = predicted_counts / predicted_counts.sum() * 100

In [ ]:
print("\nDistribution of mentalhealth posts into specific subreddits:")
for subreddit, percentage in predicted_percentages.items():
    print(f"{subreddit}: {percentage:.2f}%")

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(mentalhealth_df['confidence'], bins=20, kde=True)
plt.title('Confidence Distribution for Mentalhealth Post Classifications')
plt.xlabel('Confidence Level (Max Probability)')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

In [ ]:
high_confidence = mentalhealth_df[mentalhealth_df['confidence'] >= 0.8]
low_confidence = mentalhealth_df[mentalhealth_df['confidence'] < 0.5]

In [ ]:
print(f"\nHigh confidence predictions (>=80%): {len(high_confidence)} posts ({len(high_confidence)/len(mentalhealth_df)*100:.2f}%)")
print(f"Low confidence predictions (<50%): {len(low_confidence)} posts ({len(low_confidence)/len(mentalhealth_df)*100:.2f}%)")

In [ ]:
if len(high_confidence) > 0:
    plt.figure(figsize=(10, 6))
    sns.countplot(y='predicted_subreddit', data=high_confidence)
    plt.title('Distribution of High Confidence (>=80%) Predictions')
    plt.xlabel('Count')
    plt.ylabel('Predicted Subreddit')
    plt.tight_layout()
    plt.show()

In [ ]:
SAVE_PATH = os.path.join(os.path.dirname(os.getcwd()), 'classifications', 'transformer_classification.csv')
mentalhealth_df.to_csv(SAVE_PATH, index=False)